In [23]:
from Crypto.PublicKey import RSA
with open('no-wiener.crt') as f:
    pubkey=RSA.importKey(f.read())
n=pubkey.n
e=pubkey.e

In [24]:
e**4>n

True

In [25]:
def modular_sqrt(a, p):

    def legendre_symbol(a, p):
        """ Compute the Legendre symbol a|p using
            Euler's criterion. p is a prime, a is
            relatively prime to p (if p divides
            a, then a|p = 0)
            Returns 1 if a has a square root modulo
            p, -1 otherwise.
        """
        ls = pow(a, (p - 1) // 2, p)
        return -1 if ls == p - 1 else ls

    """ Find a quadratic residue (mod p) of 'a'. p
        must be an odd prime.
        Solve the congruence of the form:
            x^2 = a (mod p)
        And returns x. Note that p - x is also a root.
        0 is returned is no square root exists for
        these a and p.
        The Tonelli-Shanks algorithm is used (except
        for some simple cases in which the solution
        is known from an identity). This algorithm
        runs in polynomial time (unless the
        generalized Riemann hypothesis is false).
    """
    # Simple cases
    #
    if legendre_symbol(a, p) != 1:
        return 0
    elif a == 0:
        return 0
    elif p == 2:
        return p
    elif p % 4 == 3:
        return pow(a, (p + 1) // 4, p)

    # Partition p-1 to s * 2^e for an odd s (i.e.
    # reduce all the powers of 2 from p-1)
    #
    s = p - 1
    e = 0
    while s % 2 == 0:
        s //= 2
        e += 1

    # Find some 'n' with a legendre symbol n|p = -1.
    # Shouldn't take long.
    #
    n = 2
    while legendre_symbol(n, p) != -1:
        n += 1

    # Here be dragons!
    # Read the paper "Square roots from 1; 24, 51,
    # 10 to Dan Shanks" by Ezra Brown for more
    # information
    #

    # x is a guess of the square root that gets better
    # with each iteration.
    # b is the "fudge factor" - by how much we're off
    # with the guess. The invariant x^2 = ab (mod p)
    # is maintained throughout the loop.
    # g is used for successive powers of n to update
    # both a and b
    # r is the exponent - decreases with each update
    #
    x = pow(a, (s + 1) // 2, p)
    b = pow(a, s, p)
    g = pow(n, s, p)
    r = e

    while True:
        t = b
        m = 0
        for m in range(r):
            if t == 1:
                break
            t = pow(t, 2, p)

        if m == 0:
            return x

        gs = pow(g, 2 ** (r - m - 1), p)
        g = (gs * gs) % p
        x = (x * gs) % p
        b = (b * g) % p
        r = m

print(modular_sqrt(223, 17)) # should return 6

6


Attack on  Encinas-Masqu´e-Dios algorithm 

Security Analysis of an RSA Key Generation
Algorithm with a Large Private Key


solve this equation:

t=p+q, q=p/(p-1) solve p and q in Wolfram Alpha (replaced rp by p and rq by q)
i Wu
4

In [26]:
t=n%e+e
rp =  (t - modular_sqrt(t - 4,e) * modular_sqrt(t,e))//2 
rq =  (t + modular_sqrt(t - 4,e) *modular_sqrt(t,e))//2
rp%=e
rq%=e

In [37]:
rp

17124393272287076881645580791096055057192462219610193042652127026887731500987134775836063802341822881536177654071145325936609446404088362125721885987337

In [38]:
rq

28753864477129886277097526748854000254811999902065581474288880316456394728883865118963667365713504077817714302119626048090191927175530306908147361478133

In [39]:
(rp*rq)%e == n%e

True

e.nbits()=504
p approx 512 buts
q apprix 512 bits

so only (512-504)=8 bits to brute force

we take 10 bits to be sure

In [42]:
for ix in range(1024):
    for iy in range(1024):
        if (ix*e+rp)*(iy*e+rq)==n:
            x=ix
            y=iy
            print(x,y)

334 146


In [43]:

p=e*x+rp
q=e*y+rq
print(p*q==n)
d=pow(e,-1,(p-1)*(q-1))

True


In [44]:
from Crypto.PublicKey import RSA
key=RSA.construct((n,e,d))
open('private.key','wb').write(key.exportKey('PEM'))

971

In [45]:
!openssl smime -decrypt -in flag.encrypted -inkey private.key 


so{e7da1c755b398c01f30d1142a6a5d182}